In [1]:
# Imports
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import sqlite3
from sqlite3 import Error

In [2]:
# Connection to SQLite
db_file = 'mipleoDescription.db' # You need create this file on current directory
conn = None
try:
    conn = sqlite3.connect(db_file)
except Error as e:
    print(e)

In [3]:
# Creating table
c = conn.cursor()
c.execute("""
    SELECT count(name)
    FROM sqlite_master
    WHERE type='table'
    AND name='jobs'
""")

if c.fetchone()[0] == 1:
    print('Table job already exists.')
else:
    c.execute("""
        CREATE TABLE jobs(
            job_category text,
            job_name text,            
            company text,
            salary real,
            category text,
            sub_category text,
            location text,
            active_since text,
            time text,
            contract_type text,
            description text
        )
    """)
    conn.commit()

In [36]:
# Function to save records on db
def insertJobDB(conn, job):
    sql = """
        INSERT INTO jobs(
            job_category,
            job_name,
            company,
            salary,
            category,
            sub_category,
            location,
            active_since,
            time,
            contract_type,
            description
        ) VALUES (?,?,?,?,?,?,?,?,?,?,?)
    """
    c = conn.cursor()
    c.execute(sql, job)
    conn.commit()

In [37]:
# Driver
driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver')
driver.get('https://www.mipleo.com.ec/ofertas-de-trabajo/?q=')

In [38]:
# Extracting job categories
jobCategories = []
jobCategoriesNodes = driver.find_elements_by_xpath('//select[@id="category"]/option')
for e in jobCategoriesNodes:
    category = e.text
    jobCategories.append(category)

jobCategories.pop(0) # Deleting placeholder of select
print(jobCategories)

['ADMINISTRACIÓN / CONTABILIDAD / FINANZAS', 'ALMACENAMIENTO / LOGÍSTICA / DISTRIBUCIÓN', 'COMERCIAL / VENTAS / ATENCIÓN AL CLIENTE', 'DISEÑO / DECORACIÓN / ARTES GRÁFICAS', 'RECURSOS HUMANOS / RELACIONES PÚBLICAS', 'INFORMÁTICA / TELECOMUNICACIONES', 'MEDICINA / SALUD', 'PRODUCCIÓN / MANTENIMIENTO / OPERACIONES', 'MARKETING / PUBLICIDAD / PRODUCCIÓN AUDIOVISUAL', 'LEGAL / ASESORÍA', 'ARQUITECTURA / INGENIERÍAS', 'HOTELERÍA / TURISMO', 'DOCENCIA / EDUCACIÓN', 'COMPRAS / COMERCIO EXTERIOR', 'CONSTRUCCIÓN / OBRAS / EDIFICACIONES']


In [39]:
print(len(jobCategories))

15


In [40]:
SLEEP_TIME = 3 # Global variable to manage the time to await while the page is loading

In [41]:
def extractInfoSite(driver, jobCategoryName, conn):
    print(jobCategoryName)        
    cardJobsNodes = driver.find_elements_by_xpath('//div[@class="infoAd"]')    
    for i in range(len(cardJobsNodes)):
        driver.find_element_by_xpath('//div[@class="item_list"][{}]/div/span[@class="titleAd"]/a'.format(i+1)).click()
        time.sleep(SLEEP_TIME)
        try:
            jobInfo = (
                jobCategoryName,
                driver.find_element_by_xpath('//div[@class="header_item"]/h1').text, # job_name
                driver.find_element_by_xpath('//div[@class="box"][1]/span').text, # company
                driver.find_element_by_xpath('//ul[@class="info_item"]/li[1]/b').text, # salary
                driver.find_element_by_xpath('//ul[@class="info_item"]/li[2]/b').text, # category
                driver.find_element_by_xpath('//ul[@class="info_item"]/li[3]/b').text, # sub_category
                driver.find_element_by_xpath('//ul[@class="info_item"]/li[4]/b').text, # location
                driver.find_element_by_xpath('//ul[@class="info_item"]/li[5]/b').text, # active_since
                driver.find_element_by_xpath('//ul[@class="info_item"]/li[6]/b').text, # time
                driver.find_element_by_xpath('//ul[@class="info_item"]/li[7]/b').text, # contract_type
                driver.find_element_by_xpath('//div[@class="description_item"]/div[2]/p').text # description
            )
            print(jobInfo)
            insertJobDB(conn, jobInfo)
        except Exception as e:
            print(e)
        finally:
            driver.back()
            time.sleep(SLEEP_TIME)
    try:
        driver.find_element_by_xpath('//*[@id="pages"]/p/following-sibling::a').click()
        time.sleep(SLEEP_TIME)
        extractInfoSite(driver, jobCategoryName, conn)
    except NoSuchElementException:
        print('Last page, moving to the next category')
        return 1

In [42]:
def extractInfoCategory(driver, conn, jobCategoryName, jobCategoryIndex):
    # Load page by category
    driver.find_element_by_xpath('//select[@id="category"]/option[{}]'.format(jobCategoryIndex+2)).click()
    time.sleep(SLEEP_TIME)
    extractInfoSite(driver, jobCategoryName, conn)        

In [43]:
# Extracting data from DISEÑO / DECORACIÓN / ARTES GRÁFICAS
jobCategoryName = 'DISEÑO / DECORACIÓN / ARTES GRÁFICAS'
jobCategoryIndex = jobCategories.index('DISEÑO / DECORACIÓN / ARTES GRÁFICAS')
print(jobCategoryName)
print(jobCategoryIndex)

DISEÑO / DECORACIÓN / ARTES GRÁFICAS
3


In [44]:
extractInfoCategory(driver, conn, jobCategoryName, jobCategoryIndex)

DISEÑO / DECORACIÓN / ARTES GRÁFICAS
('DISEÑO / DECORACIÓN / ARTES GRÁFICAS', 'Graphic designer / Video Editor (for marketing content)', 'Heizen Ecuador', 'A convenir', 'Diseño / Decoración / Artes Gráficas', 'Diseño en General', 'Quito', '24/08/2020 - 00:10:12 am', 'Tiempo Completo', 'Contrato por tiempo indefinido', 'We are looking for a talented Graphic Designer/Video Editor to assemble recorded footage into a finished project that matches marketing vision and is suitable for broadcasting.\nUltimately, as a Video Editor, you should be able to bring sight and sound together in order to tell a cohesive story\n\nResponsibilities:\n\n- Manipulate and edit film pieces in a way that is invisible to the audience\n- Take a brief to grasp production team?s needs and specifications\n- Review shooting script and raw material to create a shot decision list based on scenes? value and contribution to continuity\n- Trim footage segments and put together the sequence of the film\n- Input music, dia

('DISEÑO / DECORACIÓN / ARTES GRÁFICAS', 'Diseñador/a (Leer primero por favor antes de aplicar)', 'ASAP CPM', '400,00 US$ (Neto mensual)', 'Diseño / Decoración / Artes Gráficas', 'Diseño en General', 'Quito', '17/08/2020 - 12:50:12 pm', 'Tiempo Completo', 'Contrato por tiempo indefinido', 'Se solicita diseñador Junior con conocimiento en impresion gran formato y manejo de maquinas de impresion, plotter de corte (manejo de cnc y corte láser será bienvenido). Solo personas que cumplan los requisitos serán tenidos en cuenta. Preferiblemente que vivan al norte de Quito. Se requieren documentos al día. Enviar CV adjunto.')
('DISEÑO / DECORACIÓN / ARTES GRÁFICAS', 'Recepción, Diseñador gráfico, web y Comunity Manager', 'Solar Ear', '400,00 US$ (Neto mensual)', 'Diseño / Decoración / Artes Gráficas', 'Diseño en General', 'Quito', '17/08/2020 - 09:15:14 am', 'Tiempo Completo', 'Contrato por tiempo indefinido', 'Buscamos Diseñador Gráfico, con los siguientes conocimientos:\nFunciones específica

('DISEÑO / DECORACIÓN / ARTES GRÁFICAS', 'Diseñador Grafico', 'MARIA FERNANDA VITERI', 'A convenir', 'Diseño / Decoración / Artes Gráficas', 'Diseño en General', 'Guayaquil', '07/08/2020 - 00:05:04 am', 'Tiempo Completo', 'Contrato por tiempo indefinido', 'Funciones:\n*Realizar el diseño de las artes para las redes sociales.\n*Proponer estrategias para captar la atención de los potenciales clientes en redes sociales.\n*Debe manejar tres rubros del negocio relacionado a electrodomésticos, servicio técnico y ropa.\n*Realizar cualquier otra función que sea designada por el jefe inmediato.\n\nCompetencias:\n*Orientación al detalle\n*Creatividad\n*Proactivo\n*Organización')
('DISEÑO / DECORACIÓN / ARTES GRÁFICAS', 'Pasante de Diseño Gráfico', 'Mercantil Makamba Cia Ltda', 'A convenir', 'Diseño / Decoración / Artes Gráficas', 'Diseño en General', 'Ambato', '07/08/2020 - 00:05:03 am', 'Beca/Prácticas', 'Contrato de Aprendizaje', 'Mercantil Makamba Cia. Ltda. es una empresa ecuatoriana con más

In [48]:
# Closing connection when finished
conn.close()

In [49]:
# Testing to prove if exists the next page
# driver.find_element_by_xpath('//*[@id="pages"]/p/following-sibling::a').click()